In [38]:
import recordlinkage as rl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import time

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
gt_train = pd.read_csv("../datasets/ground_truth/GT_train/train.csv")
gt_val = pd.read_csv("../datasets/ground_truth/GT_train/val.csv")
gt_test = pd.read_csv("../datasets/ground_truth/GT_train/test.csv")

In [10]:
df_unificato = pd.read_csv("../datasets/mediated_schema/mediated_schema_normalized.csv", dtype={'id_source_vehicles': 'object'})

/tmp/ipykernel_58916/1061716945.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unificato = pd.read_csv("../datasets/mediated_schema/mediated_schema_normalized.csv", dtype={'id_source_vehicles': 'object'})


In [13]:
# 1. Creiamo la colonna id_unificato combinando le due sorgenti
# Usiamo fillna() perché abbiamo garantito che dove manca uno c'è l'altro
df_unificato['id_unificato'] = (
    df_unificato['id_source_vehicles']
    .fillna(df_unificato['id_source_used_cars'])
)

# 2. Impostiamo l'id_unificato come INDICE del DataFrame
# Questo è il requisito fondamentale per compare.compute()
df_unificato = df_unificato.set_index('id_unificato')

In [14]:
df_unificato.head()

,id_source_vehicles,id_source_used_cars,location,price,year,manufacturer,model,cylinders,fuel_type,mileage,transmission,vin,traction,body_type,main_color,description,latitude,longitude,pubblication_date
id_unificato,,,,,,,,,,,,,,,,,,,
7316814884,7316814884,NaN,auburn,33590.0,2014.0,gmc,sierra 1500 crew cab slt,8 cylinders,gas,57923.0,other,3GTP1VEC4EG551563,NaN,pickup,white,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:18-0500
7316814758,7316814758,NaN,auburn,22590.0,2010.0,chevrolet,silverado 1500,8 cylinders,gas,71229.0,other,1GCSCSE06AZ123805,NaN,pickup,blue,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:08-0500
7316814989,7316814989,NaN,auburn,39590.0,2020.0,chevrolet,silverado 1500 crew,8 cylinders,gas,19160.0,other,3GCPWCED5LG130317,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T12:31:25-0500
7316743432,7316743432,NaN,auburn,30990.0,2017.0,toyota,tundra double cab sr,8 cylinders,gas,41124.0,other,5TFRM5F17HX120972,NaN,pickup,red,carvana is the safer way to buy a car during t...,32.590,-85.4800,2021-05-04T10:41:31-0500
7316356412,7316356412,NaN,auburn,15000.0,2013.0,ford,f150 xlt,6 cylinders,gas,128000.0,automatic,NaN,rwd,truck,black,2013 f 150 xlt v6 4 door good condition leveli...,32.592,-85.5189,2021-05-03T14:02:03-0500


In [15]:
df_unificato[df_unificato['id_source_vehicles'].isna()].head()

,id_source_vehicles,id_source_used_cars,location,price,year,manufacturer,model,cylinders,fuel_type,mileage,transmission,vin,traction,body_type,main_color,description,latitude,longitude,pubblication_date
id_unificato,,,,,,,,,,,,,,,,,,,
S2_0,NaN,S2_0,bayamon,23141.0,2019.0,jeep,renegade,i4,gasoline,7.0,a,ZACNJABB5KPJ92081,fwd,suv crossover,yellow,additional info engine 2 4l i4 zero evap m air...,18.3988,-66.1582,2019-04-06
S2_1,NaN,S2_1,san juan,46500.0,2020.0,land rover,discovery sport,i4,gasoline,8.0,a,SALCJ2FX1LH858117,awd,suv crossover,black,additional info keyless entry ebony morzine he...,18.4439,-66.0785,2020-02-15
S2_3,NaN,S2_3,san juan,67430.0,2020.0,land rover,discovery,v6,gasoline,11.0,a,SALRR2RV0L2433391,awd,suv crossover,gray,additional info fog lights 7 seat package whee...,18.4439,-66.0785,2020-02-26
S2_4,NaN,S2_4,san juan,48880.0,2020.0,land rover,discovery sport,i4,gasoline,7.0,a,SALCJ2FXXLH862327,awd,suv crossover,black,additional info keyless entry ebony morzine he...,18.4439,-66.0785,2020-04-25
S2_5,NaN,S2_5,san juan,66903.0,2020.0,land rover,range rover velar,i4,gasoline,12.0,a,SALYK2EX1LA261711,awd,suv crossover,unknown,additional info tires 21 all season adaptive d...,18.4439,-66.0785,2020-01-11


In [31]:
compare = rl.Compare()

# --- STRINGHE (Fuzzy) ---
# Usiamo Jaro-Winkler per tollerare piccoli errori di battitura
compare.string('manufacturer', 'manufacturer', method='jarowinkler', threshold=0.85, label='manufacturer')
compare.string('model', 'model', method='jarowinkler', threshold=0.85, label='model')
compare.string('location', 'location', method='jarowinkler', threshold=0.85, label='location')
compare.string('transmission', 'transmission', method='jarowinkler', threshold=0.70, label='transmission')
compare.string('cylinders', 'cylinders', method='jarowinkler', threshold=0.70, label='cylinders')
compare.string('description', 'description', method='levenshtein', threshold=0.70, label='description')

# --- ESATTI (Categorie) ---
# Se sono diversi, il punteggio è 0; se uguali è 1
compare.exact('year', 'year', label='year')
compare.exact('fuel_type', 'fuel_type', label='fuel_type')
compare.exact('traction', 'traction', label='traction')
compare.exact('body_type', 'body_type', label='body_type')
compare.exact('main_color', 'main_color', label='main_color')

# --- NUMERICI (Probabilistici) ---
# La funzione 'gauss' non dà 0/1, ma un valore tra 0 e 1 in base alla distanza
# Esempio: se il prezzo differisce di 500€ è quasi 1, se differisce di 2000€ scende drasticamente
compare.numeric('price', 'price', method='gauss', offset=500, scale=2000, label='price')
compare.numeric('mileage', 'mileage', method='gauss', offset=1000, scale=10000, label='mileage')
compare.numeric('latitude', 'latitude', method='gauss', offset=0.01, scale=0.1, label='lat')
compare.numeric('longitude', 'longitude', method='gauss', offset=0.01, scale=0.1, label='lon')

<Compare>

In [32]:
# df_unificato: il dataset mediato normalizzato (contenente record di A e B)
# gt_train, gt_val: i tuoi split della Ground Truth

# Generazione coppie di training dagli ID della GT
training_pairs = gt_train.set_index(['id_A', 'id_B']).index
val_pairs = gt_val.set_index(['id_A', 'id_B']).index

# Calcolo delle matrici di similarità (X)
# compute() legge gli ID nella GT e va a pescare i dati descrittivi nel df_unificato
print("Calcolo Feature Matrix per Training e Validation")
X_train = compare.compute(training_pairs, df_unificato, df_unificato)
X_val = compare.compute(val_pairs, df_unificato, df_unificato)

# Target vectors (y)
y_train = gt_train['label']
y_val = gt_val['label']

Calcolo Feature Matrix per Training e Validation


In [24]:
gt_train.head()

,id_A,location_A,price_A,year_A,manufacturer_A,model_A,cylinders_A,fuel_type_A,mileage_A,transmission_A,traction_A,body_type_A,main_color_A,description_A,latitude_A,longitude_A,pubblication_date_A,id_B,location_B,price_B,year_B,manufacturer_B,model_B,cylinders_B,fuel_type_B,mileage_B,transmission_B,traction_B,body_type_B,main_color_B,description_B,latitude_B,longitude_B,pubblication_date_B,label
0,S2_220894,agawam,47994.0,2020.0,ford,f150,v6,gasoline,25.0,a,4wd,pickup truck,white,you can find this 2020 ford f 150 xlt 4wd supe...,42.0889,-72.6383,2020-07-25,S2_2492412,temple,30000.0,2018.0,chevrolet,silverado 2500hd,v8 flex fuel vehicle,flex fuel vehicle,192385.0,a,4wd,pickup truck,white,awaken your sense of adventure in our 2018 che...,31.0803,-97.4114,2020-09-01,0
1,S2_1010067,pittsburgh,15995.0,2017.0,chevrolet,cruze,i4,gasoline,18353.0,a,fwd,sedan,blue,extra clean no rips no abuse no cigarette burn...,40.4958,-80.0557,2020-07-24,S2_1607148,cuba city,46686.0,2020.0,jeep,wrangler unlimited,i4,gasoline,463.0,a,4wd,suv crossover,white,additional info 1 year siriusxm guardian trial...,42.5748,-90.5510,2020-08-13,0
2,S2_187229,oneonta,29362.0,2020.0,toyota,rav4,i4,gasoline,6.0,a,awd,suv crossover,white,NaN,42.4462,-75.0118,2020-09-05,S2_2418207,grapevine,84900.0,2019.0,ford,f450 super duty,v8 biodiesel,biodiesel,16687.0,a,4wd,pickup truck,white,this ford super duty f 450 drw has a dependabl...,32.9219,-97.0634,2020-08-26,0
3,S2_2490648,navasota,34981.0,2018.0,ford,f150,NaN,NaN,14465.0,a,4x2,pickup truck,red,call today to schedule your test drive the f 1...,30.3935,-96.0711,2020-08-06,S2_2528918,albuquerque,44455.0,2020.0,ram,1500,v8,gasoline,0.0,a,4wd,pickup truck,black,additional info transmission 8 speed automatic...,35.1885,-106.6600,2020-06-19,0
4,S2_2868827,rowland heights,31325.0,2020.0,hyundai,sonata,i4,gasoline,10.0,a,fwd,sedan,white,2020 hyundai sonata sel 27 37 city highway mpg...,33.9961,-117.9220,2020-02-26,S2_700043,vienna,42909.0,2020.0,toyota,4runner,v6,gasoline,0.0,a,4wd,suv crossover,black,don t miss this great toyota packed with featu...,38.9312,-77.2434,2020-09-04,0


In [33]:
X_train.head()

,,manufacturer,model,location,transmission,cylinders,description,year,fuel_type,traction,body_type,main_color,price,mileage,lat,lon
id_A,id_B,,,,,,,,,,,,,,,
S2_220894,S2_2492412,0.0,0.0,0.0,1.0,0.0,0.0,0,0,1,1,1,9.293779e-24,5.846295e-111,0.000000e+00,0.0
S2_1010067,S2_1607148,0.0,0.0,0.0,1.0,1.0,0.0,0,1,0,0,0,2.529616e-69,1.384348e-01,1.368527e-129,0.0
S2_187229,S2_2418207,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,1,1.074931e-228,1.818805e-01,0.000000e+00,0.0
S2_2490648,S2_2528918,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,0,8.695854e-07,2.845862e-01,0.000000e+00,0.0
S2_2868827,S2_700043,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0,0,0,5.678528e-10,1.000000e+00,0.000000e+00,0.0


In [34]:
# 1. PREPARAZIONE DELLE MATRICI (Assicurati che X_train e X_val siano già calcolate)
# y_train e y_val devono contenere le label (0 o 1)

# Lista di valori per la regolarizzazione C
c_values = [0.001, 0.01, 0.1, 1, 10, 100]
# Opzioni per il peso delle classi
weights = [None, 'balanced']

best_f1 = 0
best_params = {}
best_model = None

print("--- INIZIO TUNING IPERPARAMETRI ---")
print(f"Distribuzione Training: {y_train.value_counts().to_dict()}")

start_time = time.time()

# Ciclo di Tuning
for c in c_values:
    for w in weights:
        # Inizializzazione modello
        model = LogisticRegression(C=c, class_weight=w, max_iter=1000, random_state=42)
        
        # Addestramento
        model.fit(X_train, y_train)
        
        # Predizione su Validation Set
        y_val_pred = model.predict(X_val)
        
        # Calcolo metriche
        current_f1 = f1_score(y_val, y_val_pred)
        current_prec = precision_score(y_val, y_val_pred, zero_division=0)
        current_rec = recall_score(y_val, y_val_pred, zero_division=0)
        
        print(f"C: {c:7} | Weight: {str(w):10} | F1: {current_f1:.4f} (P: {current_prec:.2f}, R: {current_rec:.2f})")
        
        # Selezione del modello migliore basata su F1-Score
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_model = model
            best_params = {'C': c, 'class_weight': w}

end_time = time.time()
tuning_duration = end_time - start_time

print("\n--- RISULTATI TUNING ---")
print(f"Miglior F1-Score su Validation: {best_f1:.4f}")
print(f"Migliori Parametri: {best_params}")
print(f"Tempo impiegato: {tuning_duration:.2f} secondi")

--- INIZIO TUNING IPERPARAMETRI ---
Distribuzione Training: {0: 5967, 1: 2983}
C:   0.001 | Weight: None       | F1: 0.9696 (P: 1.00, R: 0.95)
C:   0.001 | Weight: balanced   | F1: 0.9860 (P: 0.98, R: 0.99)
C:    0.01 | Weight: None       | F1: 0.9858 (P: 0.99, R: 0.98)
C:    0.01 | Weight: balanced   | F1: 0.9845 (P: 0.98, R: 0.99)
C:     0.1 | Weight: None       | F1: 0.9867 (P: 0.99, R: 0.98)
C:     0.1 | Weight: balanced   | F1: 0.9868 (P: 0.98, R: 0.99)
C:       1 | Weight: None       | F1: 0.9890 (P: 0.99, R: 0.99)
C:       1 | Weight: balanced   | F1: 0.9891 (P: 0.98, R: 1.00)
C:      10 | Weight: None       | F1: 0.9899 (P: 0.99, R: 0.99)
C:      10 | Weight: balanced   | F1: 0.9891 (P: 0.98, R: 0.99)
C:     100 | Weight: None       | F1: 0.9906 (P: 0.99, R: 0.99)
C:     100 | Weight: balanced   | F1: 0.9891 (P: 0.98, R: 0.99)

--- RISULTATI TUNING ---
Miglior F1-Score su Validation: 0.9906
Migliori Parametri: {'C': 100, 'class_weight': None}
Tempo impiegato: 0.25 secondi


In [39]:
# 2. VALUTAZIONE FINALE SUL TEST SET (Punto 4.H)
print("\n--- VALUTAZIONE FINALE SUL TEST SET (DATI MAI VISTI) ---")

# Generazione Feature Matrix per il Test Set
test_pairs = gt_test.set_index(['id_A', 'id_B']).index
X_test = compare.compute(test_pairs, df_unificato, df_unificato)
y_test = gt_test['label']

# Predizione finale
y_test_pred = best_model.predict(X_test)

# Report dettagliato richiesto per la consegna 4.H
print(classification_report(y_test, y_test_pred))


--- VALUTAZIONE FINALE SUL TEST SET (DATI MAI VISTI) ---
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1279
           1       1.00      0.98      0.99       639

    accuracy                           0.99      1918
   macro avg       0.99      0.99      0.99      1918
weighted avg       0.99      0.99      0.99      1918

